<a href="https://colab.research.google.com/github/Yan-Weijia/CompLing/blob/master/HW0401.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv

In [0]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [0]:
data.head(5)

,text
0,"Состояние: Отличное для возраста приставки./\nИгры отдам вместе с приставкой./\nОстальное по телефону, либо при осмотре."
1,"+5 игар фифа15,16 мартол комбат 9, гта5,асассин 4, и три диска Фифв 13 гта 4 видмак3"
2,"Продам в отличном состоянии , в комплекте проводной и беспроводной геймпады (зарядное устройство для беспроводного), гарнитура , два диска . Вообщем полный комплект, но без Kinect."
3,"Продам PS3, 2 беспроводных джойстика в комплекте. Коллекция игр будет естественно бонусом) на фото не хватает игры Mortal combat vs D.C. Джойстики не залипают,можете проверить при покупке. Обмен не интересен, так как подарили Ps4) небольшой торг )"
4,"Продаю в связи с ненадобностью.Купила для мужа ,играть не стал.Состояние идеальное .Любые проверки.Предложение идеально до 21 апреля"


In [0]:
!pip install yargy

In [0]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

# два атрибута:

In [0]:
Game = fact(
    'Game',
    ['name', 'version']
)

In [0]:
FIFA = rule(
    morph_pipeline(['FIFA', 'fifa', "фифа", 'ФИФА']).interpretation(Game.name), 
    morph_pipeline(['11', '15', '16', '17', '18', '19', '2015', '2016', '2017', '2018', '2019']).interpretation(Game.version))

In [0]:
GTA = rule(
    morph_pipeline(['GTA', 'gta', 'Grand Theft Auto', "гта", 'ГТА']).interpretation(Game.name), 
    morph_pipeline(['5', '4', 'IV', 'V', 'Chinatown Wars', 'Vice City Stories']).interpretation(Game.version)) 

In [0]:
Witcher = rule(
    morph_pipeline(['The Witcher', 'The witcher', 'the witcher', 'THE WITCHER', 'видмак', 'ВИДМАК', 'Видмак']).interpretation(Game.name), 
    morph_pipeline(['1', '2', '3', 'one', 'two', 'three']).interpretation(Game.version)) 

In [0]:
Red_dead = rule(
    morph_pipeline(['Red Dead Redemption', 'Red Dead', 'ред дед редемпшн', 'ред дед']).interpretation(Game.name), 
    morph_pipeline(['1', '2', 'one', 'two']).interpretation(Game.version)) 

In [0]:
SSF = rule(
    morph_pipeline(['Super Street Fighter', 'Street fighter', "SSF", 'супер стрит файтер']).interpretation(Game.name), 
    morph_pipeline(['I', '1', 'II', '2', 'III', '3', 'IV', '4', 'V', '5']).interpretation(Game.version))

In [0]:
GAME = or_(FIFA, GTA, Witcher, Red_dead, SSF).interpretation(Game)

parser = Parser(GAME) 

In [0]:
matches = []

for sent in data.text[:9000]:
    for match in parser.findall(sent):
        matches.append(match.fact)
print(len(matches))

921


In [0]:
print(matches[0])

Game(name='фифа', version='15')


In [0]:
for m in matches[:5]:
  print(m.name, m.version)

фифа 15
гта 5
гта 4
видмак 3
FIFA 16


извлекает только те, которые имеют оба name и version (FIFA 16, ГТА 5)

нельзя извлекать те, которые имеют только name (GTA，FIFA)

# заменить все на одно слово .const("")

In [0]:
GTA = rule(
    morph_pipeline(['GTA', 'gta', 'Grand Theft Auto', "гта", 'ГТА']).interpretation(Game.name.const("GTA")), 
    morph_pipeline(['5', '4', 'IV', 'V', 'Chinatown Wars', 'Vice City Stories']).interpretation(Game.version)) 

In [0]:
GAME = or_(GTA).interpretation(Game)

parser = Parser(GAME) 

In [0]:
matches = []

for sent in data.text[:9000]:
    for match in parser.findall(sent):
        matches.append(match.fact)
print(len(matches))

565


In [0]:
for m in matches[42:55]:
    print(m.name, m.version)

GTA vice city stories
GTA 5
GTA 5
GTA 5
GTA 5
GTA V
GTA 5
GTA 5
GTA 4
GTA 5
GTA 5
GTA IV
GTA 4


только унифицирует формы name ('GTA' вместо 'gta', 'Grand Theft Auto', "гта", 'ГТА'), 

нельзя унифицирует одинаковые версии, написанные римскими и арабскими цифрами (GTA IV, GTA 4).

# три атрибута с .optional():

In [0]:
Game = fact(
    'Game',
    ['name', 'subname','version']
)

Battlefield = rule(
    morph_pipeline(['Battlefield', 'Battlefield:', 'Батлфилд', "батлфилд", 'battlefield']).interpretation(Game.name),
    morph_pipeline(['Bad Company', '1943', 'Heroes', 'Hardline']).interpretation(Game.subname).optional(),
    morph_pipeline(['I', '1', 'II', '2', 'III', '3', 'IV', '4', 'V', '5']).interpretation(Game.version).optional()
    )

Duty = rule(
    morph_pipeline(['Call Of Duty', 'Call Of Duty:', 'Call of Duty', "сал оф дьюти", 'Сал Оф Дьюти']).interpretation(Game.name),
    morph_pipeline(['Ghosts', 'Black Ops', 'Modern Warfare', 'Modern warfare', 'MW3', 'WWII', 'World at War', 'Модерн Варфаре', 'модерн варфаре', 'варфаре', 'Варфаре']).interpretation(Game.subname).optional(),
    morph_pipeline(['I', '1', 'II', '2', 'III', '3', 'IV', '4', 'V', '5']).interpretation(Game.version).optional()
    )

Forza = rule(
    morph_pipeline(['Forza', 'форза', 'Форза', "FORZA", 'forza']).interpretation(Game.name),
    morph_pipeline(['Horizon', 'horizon', 'Motorsport', 'motorsport']).interpretation(Game.subname).optional(),
    morph_pipeline(['I', '1', 'II', '2', 'III', '3', 'IV', '4', 'V', '5', '6']).interpretation(Game.version).optional()
    )

GAME = or_(Battlefield, Duty, Forza).interpretation(Game)

parser = Parser(GAME) 

In [0]:
matches = []

for sent in data.text[:9000]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [11]:
print(len(matches))
print(matches[0])

537
Game(name='Call of Duty', subname='Ghosts', version=None)


In [13]:
for m in matches[100:105]:
    print(m.name, m.subname, m.version)

Battlefield None 4
Battlefield Hardline None
Forza MotorSport 3
Forza Motorsport 4
Forza Motorsport 4


добавлены опции (forza horizon 2, Call of Duty: Black Ops III), 

но там, где опций нет, стоит NONE (Батлфилд None None, BATTLEFIELD None 4, Battlefield Hardline None)



Уберем NONE, и теперь красиво выводит:

In [0]:
for m in matches:
  if m.subname:
    if not m.version:
      print(m.name, m.subname)
    else:
      print(m.name, m.subname, m.version)
  else:
    if not m.version:
      print(m.name)
    else:
      print(m.name, m.version)


Call of Duty Ghosts
Call of Duty 4
Call of Duty MW3
Battlefield 3
Battlefield 4
Battlefield: Bad company 2
Battlefield: Hardline
Call Of Duty: Black Ops
Call Of Duty: Modern Warfare
Battlefield 4
Battlefield 4
Battlefield 4
Call of Duty
Call of Duty
Battlefield 3
Battlefield 1
Forza Horizon 3
battlefield 4
forza
Call of Duty
battlefield 1
Call of Duty: Black Ops
Battlefield 4
battlefield 1
Battlefield Hardline
Battlefield 4
Call of duty Black ops III
Battlefield 4
Call of Duty
Forza
Call of Duty black ops
Forza horizon
Forza
battlefield 4
Battlefield Bad Company 2
Battlefield 4
Forza Motorsport 5
Call Of Duty:
Forza Motorsport 4
Battlefield 3
Call of Duty Black Ops 3
Call of Duty Black Ops 1
Call of Duty
Call of Duty
Call Of Duty: Black Ops
Battlefield 4
Battlefield
CALL OF DUTY BLACK OPS
Call of Duty Black Ops 2
Battlefield 3
call of duty world at war
Call of Duty
Battlefield: Bad Company 2
Battlefield 3
Battlefield 4
Call of duty
BattleField 1
Battlefield 4
call of duty
forza horizon